<a href="https://colab.research.google.com/github/thecrazyphysicist369/Content-Emotion-Model/blob/main/main_Roberta_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Upload the Dataset
Find the dataset **cleanData.csv** in the repository. This dataset is cleaned and pre-processed. If your dataset is not, please consider cleaning it using the **Text Preprocessing.ipynb**.*italicised text* 

In [ ]:
data = pd.read_csv("/cleanData.csv", 
                   dtype={"sentiment": "string", "content": "string"}
                   )


##Mapping the Emotions

In [ ]:
#Map the 12+ emotions in the dataset as 12
sent_to_id  = {"empty":0, 
               "sadness":1,
               "enthusiasm":2,
               "neutral":3,
               "worry":4,
                "surprise":5,
               "love":6,
               "fun":7,
               "hate":8,
               "happiness":9,
               "boredom":10,
               "relief":11,
               "anger":12}

data["sentiment_id"] = data['sentiment'].map(sent_to_id)


##Encode the dataset

In [ ]:
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(data.sentiment_id)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
Y = onehot_encoder.fit_transform(integer_encoded)

##Split the dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.clean_content,
                                                    Y, 
                                                    random_state=1995, 
                                                    test_size=0.2, 
                                                    shuffle=True)

#ROBERTA

In [ ]:
def regular_encode(texts, tokenizer, maxlen=512):
    enc_di = tokenizer.batch_encode_plus(
        texts, 
        return_attention_masks=False, 
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen
    )
    
    return np.array(enc_di['input_ids'])

def build_model(transformer, max_len=160):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(13, activation='softmax')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
MODEL = 'roberta-base'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [ ]:
X_train_t = regular_encode(X_train, tokenizer, maxlen=max_len)
X_test_t = regular_encode(X_test, tokenizer, maxlen=max_len)

In [ ]:
train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_train_t, y_train))
    .repeat()
    .shuffle(1995)
    .batch(batch_size)
    .prefetch(AUTO)
)

valid_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_test_t, y_test))
    .batch(batch_size)
    .cache()
    .prefetch(AUTO)
)


In [ ]:
transformer_layer = TFAutoModel.from_pretrained(MODEL)
model_roberta_base = build_model(transformer_layer, max_len=max_len)
model_roberta_base.summary()

In [ ]:
n_steps = X_train.shape[0] // batch_size
model_roberta_base.fit(train_dataset,steps_per_epoch=n_steps,validation_data=valid_dataset,epochs=Epoch)

###Test

In [ ]:
def get_sentiment2(model,text):
    text = clean_text(text)
    #tokenize
    x_test1 = regular_encode([text], tokenizer, maxlen=max_len)
    test1 = (tf.data.Dataset.from_tensor_slices(x_test1).batch(1))
    #test1
    sentiment = model.predict(test1,verbose = 0)
    sent = np.round(np.dot(sentiment,100).tolist(),0)[0]
    result = pd.DataFrame([sent_to_id.keys(),sent]).T
    result.columns = ["sentiment","percentage"]
    result=result[result.percentage !=0]
    return result

In [ ]:
result =get_sentiment2(model_roberta_base,"Had an absolutely brilliant day ðŸ˜ loved seeing an old friend and reminiscing")
plot_result(result)
result =get_sentiment2(model_roberta_base,"The pain my heart feels is just too much for it to bear. Nothing eases this pain. I can’t hold myself back. I really miss you")
plot_result(result)
result =get_sentiment2(model_roberta_base,"I hate this game so much,It make me angry all the time ")
plot_result(result)